# TM10007 Assignment template

In [18]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [19]:
# Data loading functions. Uncomment the one you want to use
from worcgist.load_data import load_data

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')


The number of samples: 246
The number of columns: 494


## Data orientation

In [20]:
data.head()

,label,PREDICT_original_sf_compactness_avg_2.5D,PREDICT_original_sf_compactness_std_2.5D,PREDICT_original_sf_rad_dist_avg_2.5D,PREDICT_original_sf_rad_dist_std_2.5D,PREDICT_original_sf_roughness_avg_2.5D,PREDICT_original_sf_roughness_std_2.5D,PREDICT_original_sf_convexity_avg_2.5D,PREDICT_original_sf_convexity_std_2.5D,PREDICT_original_sf_cvar_avg_2.5D,...,PREDICT_original_phasef_phasesym_median_WL3_N5,PREDICT_original_phasef_phasesym_std_WL3_N5,PREDICT_original_phasef_phasesym_skewness_WL3_N5,PREDICT_original_phasef_phasesym_kurtosis_WL3_N5,PREDICT_original_phasef_phasesym_peak_WL3_N5,PREDICT_original_phasef_phasesym_peak_position_WL3_N5,PREDICT_original_phasef_phasesym_range_WL3_N5,PREDICT_original_phasef_phasesym_energy_WL3_N5,PREDICT_original_phasef_phasesym_quartile_range_WL3_N5,PREDICT_original_phasef_phasesym_entropy_WL3_N5
ID,,,,,,,,,,,,,,,,,,,,,
GIST-001_0,GIST,0.797427,0.120594,80.192445,9.445741,7.102210,3.876595,0.927623,0.081385,0.015227,...,0.0,0.032461,7.353902,63.183181,0.0,0,0.123189,1591.133826,0.0,15.687095
GIST-002_0,non-GIST,0.896991,0.045854,35.323607,2.781343,5.339906,1.923100,0.975762,0.017550,0.009118,...,0.0,0.043431,6.452863,52.634005,0.0,0,0.172040,220.543010,0.0,12.664845
GIST-003_0,GIST,0.893238,0.051894,17.582541,1.864908,7.622507,1.039745,0.979608,0.017926,0.013258,...,0.0,0.068175,3.849719,16.277217,0.0,0,0.286710,62.150817,0.0,10.432943
GIST-004_0,non-GIST,0.914027,0.032078,21.951056,2.188093,4.757274,1.786309,0.981125,0.013734,0.011338,...,0.0,0.044148,5.725807,38.600415,0.0,0,0.182812,39.375540,0.0,10.194878
GIST-005_0,non-GIST,0.659747,0.266226,20.801224,4.025831,14.404912,7.695865,0.865275,0.171213,0.048078,...,0.0,0.043369,4.546357,23.831176,0.0,0,0.183349,24.192700,0.0,10.066832


In [21]:
data.dtypes

label                                                      object
PREDICT_original_sf_compactness_avg_2.5D                  float64
PREDICT_original_sf_compactness_std_2.5D                  float64
PREDICT_original_sf_rad_dist_avg_2.5D                     float64
PREDICT_original_sf_rad_dist_std_2.5D                     float64
                                                           ...   
PREDICT_original_phasef_phasesym_peak_position_WL3_N5       int64
PREDICT_original_phasef_phasesym_range_WL3_N5             float64
PREDICT_original_phasef_phasesym_energy_WL3_N5            float64
PREDICT_original_phasef_phasesym_quartile_range_WL3_N5    float64
PREDICT_original_phasef_phasesym_entropy_WL3_N5           float64
Length: 494, dtype: object

In [22]:
data.columns

Index(['label', 'PREDICT_original_sf_compactness_avg_2.5D',
       'PREDICT_original_sf_compactness_std_2.5D',
       'PREDICT_original_sf_rad_dist_avg_2.5D',
       'PREDICT_original_sf_rad_dist_std_2.5D',
       'PREDICT_original_sf_roughness_avg_2.5D',
       'PREDICT_original_sf_roughness_std_2.5D',
       'PREDICT_original_sf_convexity_avg_2.5D',
       'PREDICT_original_sf_convexity_std_2.5D',
       'PREDICT_original_sf_cvar_avg_2.5D',
       ...
       'PREDICT_original_phasef_phasesym_median_WL3_N5',
       'PREDICT_original_phasef_phasesym_std_WL3_N5',
       'PREDICT_original_phasef_phasesym_skewness_WL3_N5',
       'PREDICT_original_phasef_phasesym_kurtosis_WL3_N5',
       'PREDICT_original_phasef_phasesym_peak_WL3_N5',
       'PREDICT_original_phasef_phasesym_peak_position_WL3_N5',
       'PREDICT_original_phasef_phasesym_range_WL3_N5',
       'PREDICT_original_phasef_phasesym_energy_WL3_N5',
       'PREDICT_original_phasef_phasesym_quartile_range_WL3_N5',
       'PREDICT_

In [23]:
#Amount of samples
print("The amount of samples in the dataset is", len(data))

#Amount of features
print("The amount of features in the dataset is", len(data.columns))

#Amount of missing data
print("The amount of missing data in the dataset is", sum(data.isnull().sum()))

#Amount of GIST
print("The amount of gists in the dataset is", len(data[data.label=='GIST']))
#Amount of non-GIST
print("The amount of non-gists in the dataset is", len(data[data.label=='non-GIST']))

The amount of samples in the dataset is 246
The amount of features in the dataset is 494
The amount of missing data in the dataset is 0
The amount of gists in the dataset is 125
The amount of non-gists in the dataset is 121


## Missing data

In [24]:
#check if there are any missing
data.isnull().values.any()

#how many missings are there and in what columns?
data.isnull().sum()

label                                                     0
PREDICT_original_sf_compactness_avg_2.5D                  0
PREDICT_original_sf_compactness_std_2.5D                  0
PREDICT_original_sf_rad_dist_avg_2.5D                     0
PREDICT_original_sf_rad_dist_std_2.5D                     0
                                                         ..
PREDICT_original_phasef_phasesym_peak_position_WL3_N5     0
PREDICT_original_phasef_phasesym_range_WL3_N5             0
PREDICT_original_phasef_phasesym_energy_WL3_N5            0
PREDICT_original_phasef_phasesym_quartile_range_WL3_N5    0
PREDICT_original_phasef_phasesym_entropy_WL3_N5           0
Length: 494, dtype: int64

# Test & Train split

In [25]:
#split data in features and label
y = data['label']
X = data.drop(['label'], axis=1)


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Feature normalization

De meest standaard scaling gekozen van: https://scikit-learn.org/stable/modules/preprocessing.html

In [26]:
from sklearn import preprocessing

X_train_scaled = preprocessing.StandardScaler().fit_transform(X_train)

# Feature selection
## Removing features with low variance

In [27]:
from sklearn.feature_selection import VarianceThreshold
X_train_scaled_LV = VarianceThreshold(threshold=(.8 * (1 - .8)))
X_train_scaled_LV.fit_transform(X_train_scaled)

array([[-1.17608232,  2.83958564,  0.64833893, ...,  0.58798237,
        -0.36417837,  1.32229703],
       [ 0.49347027, -0.53945932,  0.8201891 , ...,  0.18061697,
        -0.36417837,  1.11564879],
       [-0.80253627,  1.57235364,  1.45048308, ..., -0.24681107,
        -0.36417837,  0.57233173],
       ...,
       [ 0.44808211, -0.50067421,  0.72377283, ...,  0.09371332,
        -0.36417837,  0.95556883],
       [-0.88331702, -0.06633901,  0.16538468, ..., -0.1321743 ,
        -0.36417837,  0.43597763],
       [-1.32805174,  1.85142973,  0.75634146, ..., -0.31234196,
        -0.36417837,  0.28900335]])

## Univariate feature selection

In [28]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

X_new = SelectKBest(f_classif, k=10).fit_transform(X_train_scaled, y_train)

/Applications/miniconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 77  83  84  90  97 103 110 467 474 475 480 487 488] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/Applications/miniconda3/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [30]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(0.9) #explain 90% variance
X_pca = pca.fit(X_train_scaled) # of fit_transform
print(X_pca.n_components_) # no. components

train_pca = X_pca.transform(X_train_scaled)
# test_pca = X_pca.transform(X_test_scaled)


43


# Models

## Linear SVC

In [ ]:
#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# RF

In [31]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Number of trees in random forest
n_estimators = [100,500,1000,2000]
# Number of features to consider at every split
max_depth = [10,30,50,80,100]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

rf_pca_tune = RandomForestClassifier()
rf_random_pca = GridSearchCV(estimator = rf_pca_tune, param_grid = random_grid, cv = 5, verbose=2, n_jobs = -1)

rf_random_pca.fit(train_pca, y_train)

best_grid_pca = rf_random_pca.best_estimator_

rf_random_pca.best_params_

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   1.6s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   1.6s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   1.6s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=500; total time=   1.6s
[CV] END max_depth=10, min_

KeyboardInterrupt: 

In [ ]:
rf_pca_tuning = RandomForestClassifier(n_estimators = 1000,max_depth= 100, min_samples_leaf=2,min_samples_split= 2, random_state = 0)
rf_pca_tuning.fit(train_pca, y_train)
pca_tune_prediction = rf_pca_tuning.predict(test_pca)
accuracy_score(y_test,pca_tune_prediction)